# 3.1 Encoder and Decoder Stacks

In [ ]:
def encoder(input_embedding):
    '''
    Composed of 6 stack of identical layers
    Each layer has two sub-layers
    
    First layer : Multi-Head Self-Attention Mechanism
    Second layer : Position-wise fully connected Feed-Forward Network.

    Residual Connection around each two sub-layer is employed, followed by layer normalization.
    The output of each sub-layer is LayerNorm (x+ Sublayer(x)) where Sublayer(x) is the function implemented by the sub-layer itself.

    To facilitate these residual connections, all sub-layers in the model (including embedding layer)
    produces outputs of dimension d_model = 512
    '''
    def ma_layer(input_embedding):
        return layer_norm
    
    def ff_layer(ma_layer_output):
        return layer_norm

    def layer_norm():
        return x + sublayer(x)
        
    def sublayer(x):
        return ########################
        
    for _ in range(6):
        ma_layer_output = ma_layer(input_embedding)
        normalized_ma_layer_output = normalize(ma_layer_output)
        
        ff_layer_output = ff_layer(normalized_ma_layer_output)
        normalized_ff_layer_output = normalize(ff_layer_output)
        input_embedding = normalized_ff_layer_output

    return normalized_ff_layer_output

In [ ]:
def decoder(encoder_output):
    '''
    Composed of 6 stack of identical layers.
    Each layer has three sub-layers.

    First layer : Masked Multi-Head Self-Attention Mechanism
    Second layer : Multi-Head Self-Attention Mechanism
    Third layer : Position-wise fully connected Feed-Forward Network.

    Residual Connection around each two sub-layer is employed, followed by layer normalization.
    The output of each sub-layer is LayerNorm (x+ Sublayer(x)) where Sublayer(x) is the function implemented by the sub-layer itself.

    To facilitate these residual connections, all sub-layers in the model (including embedding layer)
    produces outputs of dimension d_model = 512
    '''